In [1]:
import os
import numpy as np
import tensorflow as tf
from skimage.transform import resize
from skimage import morphology, measure
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
import matplotlib.pyplot as plt

In [ ]:
# 1) sanity check
print(os.getcwd())
print(os.path.exists('model_for_nuclei_segmentation.keras'))
print(glob.glob('**/*.keras', recursive=True))  # find it if you’re unsure

# 2) correct load (no leading slash)
MODEL_PATH = 'model_for_nuclei_segmentation.keras'

def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true = tf.cast(y_true, tf.float32); y_pred = tf.cast(y_pred, tf.float32)
    inter = tf.reduce_sum(y_true*y_pred, axis=[1,2,3])
    denom = tf.reduce_sum(y_true, axis=[1,2,3]) + tf.reduce_sum(y_pred, axis=[1,2,3])
    return tf.reduce_mean((2.*inter + smooth)/(denom + smooth))

def dice_loss(y_true, y_pred): return 1.0 - dice_coef(y_true, y_pred)
def bce_dice(y_true, y_pred):  return tf.keras.losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

model = tf.keras.models.load_model(
    MODEL_PATH,
    custom_objects={'dice_coef': dice_coef, 'dice_loss': dice_loss, 'bce_dice': bce_dice},
    compile=False,safe_mode=False
)


/Users/indreshmr/Image-segmentation-unet
True
['model_for_nuclei_segmentation.keras']


In [ ]:
# ---------- 2) Predict on all test images ----------
# NOTE: your model already divides by 255 inside (Lambda), so pass raw X_test.
preds_prob = model.predict(X_test, batch_size=8, verbose=1)  # shape: (N, 128, 128, 1)

# ---------- 3) Threshold ----------
THRESH = 0.5
preds_bin_128 = (preds_prob > THRESH).astype(np.uint8)

# ---------- 4) Resize masks back to original dimensions ----------
# Use nearest-neighbor (order=0) and NO anti-aliasing to keep binary masks crisp.
preds_bin_orig = []
for i in range(len(preds_bin_128)):
    h, w = sizes_test[i]  # (H, W) saved during loading
    m = resize(preds_bin_128[i, :, :, 0], (h, w),
               order=0, preserve_range=True, anti_aliasing=False).astype(np.uint8)
    preds_bin_orig.append(m)
preds_bin_orig = np.array(preds_bin_orig, dtype=np.uint8)


In [ ]:
os.makedirs('pred_masks_binary', exist_ok=True)
os.makedirs('pred_masks_instance', exist_ok=True)
for i, id_ in enumerate(test_ids):
    # Save binary mask
    plt.imsave(f'pred_masks_binary/{id_}.png', preds_bin_orig[i], cmap='gray')

    # Save instance mask with a label colormap (matshow will produce colors for each id)
    plt.figure(figsize=(4,4))
    plt.axis('off')
    plt.imshow(preds_inst_orig[i])  # integer labels; matplotlib will color them
    plt.tight_layout()
    plt.savefig(f'pred_masks_instance/{id_}.png', bbox_inches='tight', pad_inches=0)
    plt.close()
